In [166]:
import os
import numpy as np
import pandas as pd
from pprint import pprint

def open_dataset(file_name):
    return pd.read_csv(filepath_or_buffer=file_name, delimiter="\t", encoding="utf-8", header=0)

itineraries = open_dataset('data_sncf/timetables.csv')
# itineraries = open_dataset('data_sncf/test.csv')
itineraries = itineraries.sort_values(by=['trip_id', 'trajet', 'duree'], ascending=[True, True, True])

# dataframe refacto 
trajet_splited = itineraries['trajet'].str.split(' - ')

itineraries['start'] = trajet_splited.str[0]
itineraries['end'] = trajet_splited.str[1]

itineraries.drop('trajet', axis=1, inplace=True)

itineraries

,trip_id,duree,start,end
0,OCESN003100F140147152,138,Gare de Le Havre,Gare de Paris-St-Lazare
1,OCESN003190F040047309,145,Gare de Dieppe,Gare de Paris-St-Lazare
2,OCESN003198F030037315,97,Gare de Paris-St-Lazare,Gare de Rouen-Rive-Droite
3,OCESN003300F030037323,194,Gare de Cherbourg,Gare de Paris-St-Lazare
4,OCESN003313F380387526,149,Gare de Caen,Gare de Paris-St-Lazare
...,...,...,...,...
1570,OCESN895822F0500552575,244,Gare de Belfort-Ville,Gare de Lyon-Perrache
1571,OCESN895830F0200252600,103,Gare de Lons-le-Saunier,Gare de Lyon-Perrache
1572,OCESN895880F0500552634,144,Gare de Belfort-Ville,Gare de Lons-le-Saunier
1573,OCESN895940F0200252654,89,Gare de Besançon-Viotte,Gare de Lons-le-Saunier


In [167]:
# try to find a travel from id
findTravel = itineraries.loc[itineraries['trip_id'] == "OCESN014033F0900915597"]
print(findTravel)

                   trip_id  duree            start            end
19  OCESN014033F0900915597     72  Gare de Orléans  Gare de Tours


In [168]:
# Filter all possible localisations
locsStart = itineraries['start'].tolist()
locsEnd = itineraries['end'].tolist()
locs = list(dict.fromkeys(locsStart + locsEnd))
# len(locs)
locs



['Gare de Le Havre',
 'Gare de Dieppe',
 'Gare de Paris-St-Lazare',
 'Gare de Cherbourg',
 'Gare de Caen',
 'Gare de Granville',
 'Gare de Dreux',
 'Gare de Bourges',
 'Gare de Paris-Austerlitz',
 'Gare de Nevers',
 'Gare de Argentan',
 'Gare de Orléans',
 'Gare de Aubrais-(les)',
 'Gare de Paris-Bercy',
 'Gare de Calais Ville',
 'Gare de Boulogne Ville',
 'Gare de Amiens',
 'Gare de Cambrai Ville',
 'Gare de Maubeuge',
 'Gare de Paris Gare du Nord',
 'Gare de Compiègne',
 'Gare de Le Mans',
 'Gare de Nogent-le-Rotrou',
 'Gare de Chartres',
 'Gare de Blois-Chambord',
 'Gare de Lyon-Perrache',
 'Gare de Latour-de-Carol-Enveitg',
 'Gare de Font-Romeu-Odeillo-Via',
 'Gare de Briançon',
 'Gare de Romans-Bourg-de-Péage',
 'Gare de Gap',
 'Gare de Aix-en-Provence',
 'Gare de Marseille-St-Charles',
 'Gare de Les Arcs-Draguignan',
 'Gare de Grenoble',
 'Gare de Chambéry-Chal.-les-Eaux',
 'Gare de Annecy',
 'Gare de Avignon-Centre',
 'Gare de Lyon-Part-Dieu',
 'Gare de Dijon-Ville',
 'Gare de M

In [169]:
# get all possible itineraries from un localisation
def get_itineraries(fromLoc):
    return itineraries.loc[itineraries['start'] == fromLoc]


In [170]:
# update distance of localisation from source
def updateDistanceFromSource(matrix, itineraries):
    # print(itineraries)
    
    for index, row in itineraries.iterrows():
        
        # if initerary is not yet processed
        if matrix[row['end']]['processed'] == False:
        
            oldValue = matrix[row['end']]['distanceFromSource']
            newValue = matrix[row['start']]['distanceFromSource'] + row['duree']
            
            if oldValue == 'infinite':
                matrix[row['end']]['distanceFromSource'] = newValue
                matrix[row['end']]['fromLoc'] = row['start']
            else:   
                if newValue < oldValue:
                    matrix[row['end']]['distanceFromSource'] = newValue
                    matrix[row['end']]['fromLoc'] = row['start']
        
        
    print("-----------------------------------------------------------------")
    print("---------- MISE A JOUR DES DISTANCES DEPUIS LA SOURCE -----------")
    print("-----------------------------------------------------------------")
    print(" ")
    
    return matrix

In [171]:
# Find the minimum distance that has not been processed and that is not infinite
def findMiniDistanceFromSource(matrix):
    
    toProcess = {}
    
    for index, row in matrix.items():
        if row['processed'] == False and row['distanceFromSource'] != 'infinite':
            toProcess[index] = row
            
    print("-------------------- Les itineraires pas encore traités ---------------------")
    print(" ")
    [print(key,':',value) for key, value in toProcess.items()]
    print(" ")
    
    miniValue = 9999999999
    location = ""
    
    for index, row in toProcess.items():
    
        if row['distanceFromSource'] < miniValue:
            miniValue = row['distanceFromSource']
            location = index
            
       
    if location != "":    
        print("-------------------- La distance la plus courte pas encore traitée ---------------------")
        print(" ")
        print("Etape : ", location, " - Distance depuis la source : ", miniValue)
        print(" ")
    else:
        print("------------------------------------------------------------------------")
        print("-------------------- Plus de trajets lié à traiter ---------------------")
        print("------------------------------------------------------------------------")
        print(" ")
        
    return location
    
                
    

In [172]:
# find the shortest distance between source and destination 
def get_best_itinerary(matrix, source, destination):
    toProcess = {}
    distance = ""
    fromLoc = ""
    itineraries = []
    
    for index, row in matrix.items():
        if row['processed'] == True and row['distanceFromSource'] != 'infinite':
            toProcess[index] = row
            
    # print(toProcess)
    if destination in toProcess.keys():
        print(toProcess[destination])
        distance = toProcess[destination]["distanceFromSource"]
        fromLoc = toProcess[destination]["fromLoc"]
        itineraries.insert(0, destination)
        
        while fromLoc != source:
            itineraries.insert(0, fromLoc)
            fromLoc = toProcess[fromLoc]["fromLoc"]
            
        itineraries.insert(0, source)
        
        return {
            'itineraries': itineraries,
            'distance': distance
        }
    
    else:
        print("-----------------------------------------------------------------")
        print("-------------------- Ce trajet n'existe pas ---------------------")
        print("-----------------------------------------------------------------")
        print("")
        
        return "None"

In [173]:
# Find the shortest path among all possible sources and destinations 
def dijsktra(df, source, destination):
    matrixLoc = {}
    
    locsStart = df['start'].tolist()
    locsEnd = df['end'].tolist()
    locs = list(dict.fromkeys(locsStart + locsEnd))
    
    # filter possible localisations
    locsStart = list(filter(lambda k: source in k, locs))
    locsEnd = list(filter(lambda k: destination in k, locs))
    
    # print(locsStart)
    # print(locsEnd)
    
    returnResult = "None"

    for locStart in locsStart:
        matrixLoc = {}
        
        for locEnd in locsEnd:
            if locEnd == locStart:
                continue
            
            matrixLoc = {}
    
            print("Trajet : ", locStart, " - ", locEnd)
            print(" ")
            
            # initialize matrix
            for loc in locs:
                matrixLoc[loc] = {
                    'distanceFromSource': 'infinite',
                    'fromLoc': '',
                    'processed': False,
                }
                
            # initialize matrix source values
            matrixLoc[locStart]['distanceFromSource'] = 0
            matrixLoc[locStart]['fromLoc'] = locStart
            matrixLoc[locStart]['processed'] = True
            
            nextLocation = locStart
            
            while nextLocation != "":

                if not get_itineraries(nextLocation).empty:
                    matrixLoc = updateDistanceFromSource(matrixLoc, get_itineraries(nextLocation))
                else:
                    print("Aucun itinéraire en partant de ", nextLocation)
                    print(" ")
                    
                nextLocation = findMiniDistanceFromSource(matrixLoc)

                if nextLocation != "":
                    matrixLoc[nextLocation]['processed'] = True
            
            result = get_best_itinerary(matrixLoc, locStart, locEnd)
            
            if result != "None":
                return result

    return "None"
    

In [198]:
def get_itinerary(df, source, destination):
    result = dijsktra(df, source, destination)

    if result == "None":
        result = dijsktra(df, destination, source)

    if result != "None":
        itineraries = result['itineraries']
        itineraries.reverse()
        result['itineraries'] = itineraries
        return result
    
    else:
        print("-----------------------------------------------------------------")
        print("---------------------- Aucun trajet trouvé ----------------------")
        print("-----------------------------------------------------------------")
        print("")
        return
    

In [200]:
locStart = "Paris"
locEnd = "Bordeaux"

get_itinerary(itineraries, locStart, locEnd)


Trajet :  Gare de Paris-St-Lazare  -  Gare de Bordeaux-St-Jean
 
-----------------------------------------------------------------
---------- MISE A JOUR DES DISTANCES DEPUIS LA SOURCE -----------
-----------------------------------------------------------------
 
-------------------- Les itineraires pas encore traités ---------------------
 
Gare de Rouen-Rive-Droite : {'distanceFromSource': 97, 'fromLoc': 'Gare de Paris-St-Lazare', 'processed': False}
Gare de Trouville-Deauville : {'distanceFromSource': 147, 'fromLoc': 'Gare de Paris-St-Lazare', 'processed': False}
Gare de Vernon-Giverny : {'distanceFromSource': 60, 'fromLoc': 'Gare de Paris-St-Lazare', 'processed': False}
Gare de Serquigny : {'distanceFromSource': 105, 'fromLoc': 'Gare de Paris-St-Lazare', 'processed': False}
 
-------------------- La distance la plus courte pas encore traitée ---------------------
 
Etape :  Gare de Vernon-Giverny  - Distance depuis la source :  60
 
Aucun itinéraire en partant de  Gare de Vernon-G

{'itineraries': ['Gare de Paris-Austerlitz',
  'Gare de Orléans',
  'Gare de Limoges-Bénédictins',
  'Gare de Bordeaux-St-Jean'],
 'distance': 423}